In [25]:
import cv2
import matplotlib.pyplot as plt
import os
import natsort
import shutil
import re
import numpy as np

%matplotlib inline

# img_path ='./sample/image/0000/000094.png'
# image = cv2.imread(img_path)

In [26]:

base_path = "/mnt/nas3/Data/kitti-processed/object_tracking/training"


calib_path = os.path.join(base_path,"calib")
image_path = os.path.join(base_path,"image_02")
label_path = os.path.join(base_path,"label_02")
lidar_path = os.path.join(base_path,"velodyne")


output_base_path = "/home/eslim/workspace/tracking_convert/training"
output_lidar_path = os.path.join(output_base_path, "points")
output_label_path = os.path.join(output_base_path, "label")

# output_image_path = os.path.join(output_base_path, "image_02")
# output_calib_path = os.path.join(output_base_path, "calib")


In [27]:
def convert_bin_to_npy(file_path, save_path):
    arr = np.fromfile(file_path, dtype=np.float32)
    np.save(save_path, arr)


def read_calib(calib_path):
    with open(calib_path) as f:
        for line in f.readlines():
            if line[:2] == "P2":
                P2 = re.split(" ", line.strip())
                P2 = np.array(P2[-12:], np.float32)
                P2 = P2.reshape((3, 4))
            if line[:14] == "Tr_velo_to_cam" or line[:11] == "Tr_velo_cam":
                vtc_mat = re.split(" ", line.strip())
                vtc_mat = np.array(vtc_mat[-12:], np.float32)
                vtc_mat = vtc_mat.reshape((3, 4))
                vtc_mat = np.concatenate([vtc_mat, [[0, 0, 0, 1]]])
            if line[:7] == "R0_rect" or line[:6] == "R_rect":
                R0 = re.split(" ", line.strip())
                R0 = np.array(R0[-9:], np.float32)
                R0 = R0.reshape((3, 3))
                R0 = np.concatenate([R0, [[0], [0], [0]]], -1)
                R0 = np.concatenate([R0, [[0, 0, 0, 1]]])
    vtc_mat = np.matmul(R0, vtc_mat)
    return (P2, vtc_mat)
    

In [28]:
def fov_filter(src_file_path,frame_idx,calib_path):
    P2, V2C = read_calib(
            os.path.join(calib_path, f"{str(frame_idx).zfill(4)}.txt")
    )
    max_row = 374  # y
    max_col = 1241  # x
    
    lidar = np.fromfile(src_file_path, dtype=np.float32).reshape(
        -1, 4
    )
    
    mask = lidar[:, 0] > 0
    lidar = lidar[mask]
    
    lidar_copy = np.zeros(shape=lidar.shape)
    lidar_copy[:, :] = lidar[:, :]
    velo_tocam = V2C
    lidar[:, 3] = 1
    lidar = np.matmul(lidar, velo_tocam.T)
    img_pts = np.matmul(lidar, P2.T)
    
    velo_tocam = np.mat(velo_tocam).I
    velo_tocam = np.array(velo_tocam)
    normal = velo_tocam
    normal = normal[0:3, 0:4]
    lidar = np.matmul(lidar, normal.T)
    lidar_copy[:, 0:3] = lidar
    x, y = img_pts[:, 0] / img_pts[:, 2], img_pts[:, 1] / img_pts[:, 2]
    mask = np.logical_and(
        np.logical_and(x >= 0, x < max_col), np.logical_and(y >= 0, y < max_row)
    )
    points = lidar_copy[mask]
    return points

In [29]:
def copy_files(src_file, dest_folder,dest_file_name):
    # 대상 폴더가 존재하지 않으면 생성
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    # 소스 폴더 내의 모든 파일 복사
  
    # src_file = os.path.join(src_folder, filename)
    dest_file = os.path.join(dest_folder, dest_file_name)
        
        # 파일인지 확인 (폴더는 복사하지 않음)
    if os.path.isfile(src_file):
        shutil.copy(src_file, dest_file)
        # print(f"Copied: {src_file} to {dest_file}")

def save_files(points, dest_folder,dest_file_name):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    # 소스 폴더 내의 모든 파일 복사
  
    # src_file = os.path.join(src_folder, filename)
    dest_file = os.path.join(dest_folder, dest_file_name)
    np.save(dest_file, points)

       


In [30]:
# lidar

lidar_dir_list = os.listdir(lidar_path)
lidar_dir_list = natsort.natsorted(lidar_dir_list)
lidar_dir_list
last_name =0
for idx,lidar_dir in enumerate(lidar_dir_list):
    files = os.listdir(os.path.join(lidar_path, lidar_dir))
    files = natsort.natsorted(files)

    for file in files:
        # def fov_filter(src_file_path,frame_idx,calib_path):
        src_file_path = os.path.join(lidar_path, lidar_dir, file)
        points = fov_filter(src_file_path, idx,calib_path)
        save_files(points, output_lidar_path, str(int(file.split(".")[0]) + last_name).zfill(6)+".npy")
        # copy_files(os.path.join(lidar_path, lidar_dir, file),output_lidar_path, str(int(file.split(".")[0]) + last_name).zfill(6)+".bin")
    last_name = files[-1]
    last_name = int(last_name.split(".")[0]) if type(last_name)  != int  else 0
    print(idx)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [31]:
point = np.load("/home/eslim/workspace/tracking_convert/training/points/000000.npy")
point

array([[ 2.27189999e+01,  3.09999995e-02,  9.76999998e-01,
         3.19999993e-01],
       [ 1.80520000e+01,  7.59999976e-02,  8.19999993e-01,
         4.60000008e-01],
       [ 1.80259991e+01,  1.31999999e-01,  8.19000006e-01,
         2.39999995e-01],
       ...,
       [ 6.62300014e+00, -3.40000018e-02, -1.73500001e+00,
         2.00000003e-01],
       [ 6.62500000e+00, -2.30000000e-02, -1.73599994e+00,
         2.50000000e-01],
       [ 6.62699986e+00, -3.00000003e-03, -1.73599994e+00,
         2.30000004e-01]])

In [182]:
#label
def load_label(path):
    with open(path, 'r') as file:
        lines = file.read().splitlines()
    return lines
    

In [183]:
lines = load_label(os.path.join(label_path,"0000.txt"))
line = lines[2]
line
# line = line.split()
# words = [word for line in content.splitlines() for word in line.split()]
# line
# line

'0 0 Van 0 0 -1.793451 296.744956 161.752147 455.226042 292.372804 2.000000 1.823255 4.433886 -4.552284 1.858523 13.410495 -2.115488'

In [184]:
def cam_to_lidar_pose(box, vtc_mat):
    box.append(1)
    mat = np.array(box)
    ctv_mat = vtc_mat.T    
    mat = mat.reshape(4,1)
    mat = mat.astype('float64')
    transformed_mat = ctv_mat @ mat
    T = np.array(transformed_mat.T, dtype=np.float32)
    return T

In [185]:
def save_label( P2,V2C,label_list, dest_path):
    class_name = label_list[2]
    dim = np.array(label_list[10:13]).astype('float64')
    dim = dim[::-1]
    location = label_list[13:16]
    rot = float(label_list[-1])
    location_lidar = cam_to_lidar_pose(location, V2C)[0]
    rot_lidar = -rot - np.pi/2
    location_lidar[2] += dim[2]/2
    
    if os.path.exists(dest_path):
        with open(dest_path, "a") as f:
            f.write(f"{location_lidar[0]:.6f} {location_lidar[1]:.6f} {location_lidar[2]:.6f} {dim[0]:.6f} {dim[1]:.6f} {dim[2]:.6f} {rot_lidar:.6f} {class_name}\n")
    else:
        with open(dest_path, "w") as f:
            f.write(f"{location_lidar[0]:.6f} {location_lidar[1]:.6f} {location_lidar[2]:.6f} {dim[0]:.6f} {dim[1]:.6f} {dim[2]:.6f} {rot_lidar:.6f} {class_name}\n")

        
            

In [186]:
# lines[0]

In [ ]:
# load_label(label_path)

label_dir_list = os.listdir(label_path)
label_dir_list = natsort.natsorted(label_dir_list)
# lidar_dir_list
last_name =0
last_idx=0



if not os.path.exists(output_label_path):
    os.makedirs(output_label_path)

for idx, label_dir in enumerate(label_dir_list):
    # print(os.path.join(label_path,label_dir))
    lines = load_label(os.path.join(label_path,label_dir))
    # print(lines)
    for line in lines:
        line = line.split()
        frame_idx = int(line[0]) + last_idx
        if line[2] != "Pedestrian" and line[2] != "Car" and line[2] != "Cyclist":
            continue
        
        # print(frame_idx)
        
        P2, V2C = read_calib(os.path.join(calib_path,f"{str(idx).zfill(4)}.txt"))
        save_label( P2,V2C,line,os.path.join(output_label_path,f"{str(frame_idx).zfill(6)}.txt"))
        # break
        # last_idx = int(line[0])
    last_idx = frame_idx
        # print(line)
        # print("check")


In [175]:
# # image

# image_dir_list = os.listdir(image_path)
# image_dir_list = natsort.natsorted(image_dir_list)
# image_dir_list
# last_name =0
# for idx,image_dir in enumerate(image_dir_list):
#     files = os.listdir(os.path.join(image_path, image_dir))
#     files = natsort.natsorted(files)
#     for file in files:
#         copy_files(os.path.join(image_path, image_dir, file),output_image_path, str(int(file.split(".")[0]) + last_name).zfill(6)+".png")
#     last_name = files[-1]
#     last_name = int(last_name.split(".")[0]) if type(last_name)  != int  else 0
#     print(idx)

In [ ]:
# # calib
# image_dir_list = os.listdir(image_path)
# image_dir_list = natsort.natsorted(image_dir_list)

# calib_dir_list = os.listdir(calib_path)
# calib_dir_list = natsort.natsorted(calib_dir_list)

# last_idx = 0
# for idx, image_dir in enumerate(image_dir_list):
#     files = os.listdir(os.path.join(image_path, image_dir))
#     # files = natsort.natsorted(files)
#     last_name = len(files)
#     # last_name = int(last_name.split(".")[0])
#     for name in range(last_name):
#         src_calib_path = os.path.join(calib_path, calib_dir_list[idx])
#         dst_calib_name = str(last_idx+name).zfill(6)
#         copy_files(src_calib_path, output_calib_path, dst_calib_name+".txt")
#     last_idx = last_name
#     print(idx)




In [160]:
arr = np.array([1,2,3,4,5])

arr = arr[::-1]
arr

array([5, 4, 3, 2, 1])